In [1]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalizationV2

In [2]:
train_path = '../data/black_box_single/seg_train/'
test_path = '../../normal/archive/seg_test/'

In [3]:
train_datagen = ImageDataGenerator(
                    rescale = 1/255,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True,
                    vertical_flip = False)
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (64, 64),
                                                 class_mode = 'categorical')

Found 14034 images belonging to 6 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale = 1/255)
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (64, 64),
                                            class_mode = 'categorical')

Found 3000 images belonging to 6 classes.


In [5]:
cnn = Sequential()
cnn

Metal device set to: Apple M1


2022-08-29 19:20:20.807428: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-29 19:20:20.807596: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
cnn.add(Convolution2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Convolution2D(filters=256, kernel_size=3, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Dropout(0.4)),

cnn.add(Flatten())

cnn.add(Dense(units=256, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=128, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=64, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=24, activation='relu'))
cnn.add(BatchNormalizationV2())
cnn.add(Dropout(0.25))

cnn.add(Dense(units=6, activation='softmax'))

In [7]:
cnn.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [8]:
%%time
cnn.fit(training_set, steps_per_epoch= len(training_set), validation_data = test_set, validation_steps=len(test_set), epochs = 10)

Epoch 1/17


2022-08-29 19:20:21.438032: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-29 19:20:22.569800: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


439/439 [==============================] - ETA: 0s - loss: 1.6264 - accuracy: 0.3747

2022-08-29 19:20:57.226476: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


439/439 [==============================] - 38s 83ms/step - loss: 1.6264 - accuracy: 0.3747 - val_loss: 1.2238 - val_accuracy: 0.5297
Epoch 2/17
439/439 [==============================] - 31s 72ms/step - loss: 1.1878 - accuracy: 0.5199 - val_loss: 1.0272 - val_accuracy: 0.6090
Epoch 3/17
439/439 [==============================] - 32s 73ms/step - loss: 1.0800 - accuracy: 0.5754 - val_loss: 0.8859 - val_accuracy: 0.6717
Epoch 4/17
439/439 [==============================] - 31s 71ms/step - loss: 1.0013 - accuracy: 0.6158 - val_loss: 0.9276 - val_accuracy: 0.6427
Epoch 5/17
439/439 [==============================] - 31s 70ms/step - loss: 0.9552 - accuracy: 0.6416 - val_loss: 1.0382 - val_accuracy: 0.6070
Epoch 6/17
439/439 [==============================] - 31s 70ms/step - loss: 0.9055 - accuracy: 0.6667 - val_loss: 1.1082 - val_accuracy: 0.5890
Epoch 7/17
439/439 [==============================] - 30s 69ms/step - loss: 0.8698 - accuracy: 0.6817 - val_loss: 0.7445 - val_accuracy: 0.7327
Epo

In [20]:
accuracy=cnn.evaluate(test_set, steps=len(test_set),verbose=2)
print('Accuracy of the model on the test set: ',accuracy[1])

2/2 - 0s - loss: 0.5683 - accuracy: 0.7812 - 79ms/epoch - 40ms/step
Accuracy of the model on the test set:  0.78125
